**Nicolly Zorzam Moura**

DRE: 121037550

# O problema:
Atualmente a indústria cinematográfica é extremamente diversificada e, dessa forma, há praticamente infinitas possibilidades de filmes para assistir com uma companhia numa tarde de domingo. Apesar de parecer um cenário bastante
satisfatório, normalmente uma problemática permeia esse meio: a dificuldade de
selecionar um filme específico dentre tantas opções existentes, situação que se agrava ainda mais quando tratamos da escolha de um filme para mais de uma pessoa, pois temos que levar em conta as preferências de cada uma.

Sendo este um problema recorrente no cotidiano das pessoas, as plataformas de streaming desenvolveram Sistemas de Recomendação, cujo objetivo é justamente auxiliar nessa tomada de decisão. Porém, obviamente sendo do próprio interesse dessas empresas propiciar a permanência do usuário em sua plataforma, as sugestões incluem somente os filmes que encontram-se disponíveis naquele serviço de streaming em específico, descartando outras produções que não podem ser buscadas ali. Como consequência, temos a “perda” de recomendações que poderiam ser tão boas quanto, ou até melhores do que, as sugestões apresentadas. Pensando nisso, e também levando em consideração que é muito mais difícil essa tomada de decisão quando envolve mais de uma pessoa (serviço que não é ofertado pelas plataformas de streaming), optei por me aprofundar no estudo acerca de Sistemas de Recomendação, criando um protótipo em que fosse possível a recomendação de filmes para mais de uma pessoa, usando dados da plataforma *MovieLens* (https://grouplens.org/datasets/movielens/), para que as sugestões englobassem todas as plataformas de streaming disponíveis.


In [ ]:
import pandas as pd # Biblioteca que será usadas para manipular os dataframes obtidos a partir dos dados do MovieLens
import numpy as np
import scipy as sp
import scipy.linalg
import math
from IPython.display import clear_output # Apenas para limpar a tela

In [ ]:
# Coleta de dados do MovieLens
url_movies = "https://raw.githubusercontent.com/NicZorzam/Projeto_ALA/main/movies.csv"
filmes = pd.read_csv(url_movies)
filmes.columns = ["filmeID", "titulo", "generos"]
filmes = filmes.set_index("filmeID")
filmes.head() # Mostra os primeiros 5 filmes

,titulo,generos
filmeID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Coleta de dados do MovieLens
url_ratings = "https://raw.githubusercontent.com/NicZorzam/Projeto_ALA/main/ratings.csv"
notas = pd.read_csv(url_ratings)
notas.columns = ["usuarioID", "filmeID", "nota", "momento"]
notas.head() # Mostra as primeiras 5 notas

,usuarioID,filmeID,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
total_de_votos = notas["filmeID"].value_counts()
filmes["total de votos"] = total_de_votos

In [ ]:
notas_medias = notas.groupby("filmeID").mean()["nota"]
filmes["nota media"] = notas_medias

# Primeiro passo: coleta de dados
O principal objetivo deste sistema de recomendação é basear itens pelo feedback de outros itens já experienciados pelos usuários. Dessa forma, antes de tudo, é necessário conhecermos mais acerca do gosto pessoal de cada um dos usuários do sistema. Então, o protótipo inicia a interação com o usuário pedindo para que ele avalie alguns filmes famosos com uma nota que varia de 0.5 até 5, sendo possível as notas 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, ou N caso não tenha assistido ao filme. Assim, as notas dadas são colocadas numa lista [id_filme, nota_filme], dentro de uma lista de filmes, como, por exemplo:


```
ListaFilmes_Usuario1 = [
    [id_filme1, nota_filme1],
    [id_filme2, nota_filme2],
]
```

Mais tarde, essa lista será utilizada para comparar com os dados cedidos pelo MovieLens para calcular o melhor filme para os usuários do sistema.

Importante ressaltar que, para a avaliação de cada filme, cada usuário deverá classificar com uma nota separadamente, sendo solicitado primeiro a nota do usuário 1 e depois a nota do usuário 2.


In [ ]:
# Nomeando os usuários: (com o objetivo de tornar a experiência mais pessoal)
usuario_1 = input("Qual é o nome do primeiro usuário? ")
usuario_2 = input("Qual é o nome do segundo usuário? ")

Qual é o nome do primeiro usuário? julia
Qual é o nome do segundo usuário? davi


In [ ]:
def Classificação_usuario(usuario, filme):
  classificações_possíveis = ["0.5", "1", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5", "N"]

  while True:
    nota_usuario = input(f"Nota {usuario}: ")
    if nota_usuario in classificações_possíveis:
      return nota_usuario
    print("Por favor, insira uma resposta válida:")

In [ ]:
def Classificando_filmes(filme, usuario_1, usuario_2):

  print(f"Avalie o filme {filme} com uma nota de 0,5 a 5:")
  print("\n")
  print("ATENÇÃO as notas possíveis são: 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5.")
  print(f"Caso não tenha assistido {filme}, por favor insira 'N'.")
  print("\n")

  nota_usuario_1 = Classificação_usuario(usuario_1, filme)
  if nota_usuario_1 == 'N':
    nota_usuario_1 = np.NaN
  else: nota_usuario_1 = float(nota_usuario_1)

  nota_usuario_2 = Classificação_usuario(usuario_2, filme)
  if nota_usuario_2 == 'N':
    nota_usuario_2 = np.NaN
  else: nota_usuario_2 = float(nota_usuario_2)

  clear_output(wait = True)
  return nota_usuario_1, nota_usuario_2

In [ ]:
# Função para achar o ID de cada filme a ser avaliado (lista_filmes)
def filmeIndex(titulo):
  filme_index = filmes.index[filmes["titulo"] == titulo].tolist()
  filme_index = filme_index[0]

  return filme_index

In [ ]:
# Lista de filmes a ser avaliada pelos usuários (filmes previamente selecionados. Formato de nome devido ao dataframe do MovieLens)
lista_filmes = ["Godfather, The (1972)", "Wizard of Oz, The (1939)", "Rocky (1976)", "Up (2009)", "Braveheart (1995)",
                "Lord of the Rings: The Fellowship of the Ring, The (2001)", "Avatar (2009)", "Twilight (2008)", "West Side Story (1961)",
                "Lion King, The (1994)", "Titanic (1997)", "Toy Story (1995)", "Matrix, The (1999)", "Alien (1979)", "Shining, The (1980)", "Fight Club (1999)"]

In [ ]:
# Função que cria a estrutura básica de cada lista de dados do usuário
def Filmes_Nota(lista_filmes):
  lista_filmesID = []

  for filme in lista_filmes:
    lista_filmesID.append([filmeIndex(filme), np.NaN])

  return lista_filmesID

In [ ]:
def Classificação(lista_filmes, usuario_1, usuario_2):
  lista_filmesBR = ["O Poderoso Chefão (1972)", "O Mágico de Oz (1939)", "Rocky: Um Lutador (1976)", "Up - Altas Aventuras (2009)", "Coração Valente (1995)",
                    "O Senhor dos Anéis: A Sociedade do Anel (2001)", "Avatar (2009)", "Crepúsculo (2008)", "Amor, Sublime Amor (1961)", "O Rei Leão (1994)",
                    "Titanic (1998)", "Toy Story – Um Mundo de Aventuras (1995)", "Matrix (1999)", "Alien - O 8º Passageiro (1979)", "O Iluminado (1980)",
                    "Clube da Luta (1999)"]

  filmes_não_assistidos = []

  nota_usuario_1 = Filmes_Nota(lista_filmes)
  nota_usuario_2 = Filmes_Nota(lista_filmes)

  x = 0
  for filme in lista_filmesBR:
    nota1, nota2 = Classificando_filmes(filme, usuario_1, usuario_2)
    nota_usuario_1[x][1] = nota1
    if nota1 is np.NaN:
      if nota_usuario_1[x][0] not in filmes_não_assistidos:
        filmes_não_assistidos.append(nota_usuario_1[x][0])

    nota_usuario_2[x][1] = nota2
    if nota2 is np.NaN:
      if nota_usuario_1[x][0] not in filmes_não_assistidos:
        filmes_não_assistidos.append(nota_usuario_2[x][0])

    x+=1

  return nota_usuario_1, nota_usuario_2, filmes_não_assistidos

In [ ]:
dados_usuario1, dados_usuario2, não_assistidos = Classificação(lista_filmes, usuario_1, usuario_2)

Avalie o filme Clube da Luta (1999) com uma nota de 0,5 a 5:


ATENÇÃO as notas possíveis são: 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5.
Caso não tenha assistido Clube da Luta (1999), por favor insira 'N'.


Nota julia: 5
Nota davi: 5


In [ ]:
def novo_usuario(dados, usuarioID):
  notas_usuario = pd.DataFrame(dados, columns=["filmeID", "nota"]) # Transformando em dataframe
  notas_usuario['usuarioID'] = usuarioID
  return pd.concat([notas, notas_usuario]) # Concatenando as duas dataframe


In [ ]:
usuario1ID = notas['usuarioID'].max()+1 # Definindo o ID do novo usuario como o primeiro sucessor do ultimo usuario
usuario2ID = notas['usuarioID'].max()+2

In [ ]:
notas = novo_usuario(dados_usuario1, usuario1ID) # Atualizando o data frame

In [ ]:
notas = novo_usuario(dados_usuario2, usuario2ID) # Atualizando o data frame

In [ ]:
notas.tail(32) # Mostrando os 32 ultimos elementos do data frame e conferindo as notas colocadas

,usuarioID,filmeID,nota,momento
0,611,858,5.0,NaN
1,611,919,3.0,NaN
2,611,1954,3.0,NaN
3,611,68954,2.0,NaN
4,611,110,2.0,NaN
5,611,4993,1.5,NaN
6,611,72998,1.0,NaN
7,611,63992,0.5,NaN
8,611,1947,NaN,NaN
9,611,364,1.0,NaN


# Analisando os dados coletados:
Após coletarmos os dados e os colocarmos no dataframe "notas" com seus valores, é possível analisarmos essas informações obtidas. Uma implementação que seria interessante haver no protótipo é uma função que, de alguma forma, calculasse a similaridade entre esses dois usuários. Pensando nisso, implementei o cálculo da distância entre esses dois vetores, em que ele devolve uma porcentagem que estima a similaridade das preferências dos usuários. Optei pela distância por achar um meio “mais direto” e que, nesse contexto, se encaixaria melhor, diferentemente do cálculo do cosseno do ângulo entre os vetores, que poderia retornar um valor completamente impreciso. Por exemplo, num cenário hipotético que o usuário 1 tenha avaliado todos os filmes com a nota 5, e o usuário 2 avaliasse todos os filmes com a nota 0.5, eles teriam cosseno = 1, ainda que os gostos sejam completamente diferentes.

Dessa forma, seguindo com o cálculo de distância, é necessário entendermos que os usuários devem ter um número mínimo de filmes que ambos assistiram para que o cálculo possa ser feito. Assim, comparando apenas estes filmes assistidos por ambos, caso a distância = 0, a função retorna 100% de similaridade entre os usuários. Por outro lado, caso
`distancia = math.sqrt(math.pow(4.5, 2) * n) `
sendo n o número de filmes que ambos assistiram, a similaridade retornada é igual a 0%. Esse cálculo vem da própria fórmula da distância e representa o pior caso de todos: quando a diferença de nota de todos os filmes é igual a 4.5 (a maior possível). Por fim, com esses valores, é possível calcular qualquer porcentagem.


In [ ]:
# Extraindo notas de usuarios
def notas_do_usuario(usuario):
  notas_usuario = notas.query("usuarioID==%d" % usuario) # apenas notas desse usuario de todos os filmes avaliados por ele
  notas_usuario = notas_usuario[["filmeID", "nota"]] # "retirar" as colunas usuarioID e momento da matriz
  notas_usuario = notas_usuario.set_index("filmeID") # setar o index pelos filmes, retira a coluna index

  return notas_usuario

In [ ]:
# Função que une em um unico data frame as notas dos usuarios 1 e 2
def diferencas(usuario_id1, usuario_id2):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  diferencas = notas1.join(notas2, lsuffix="_esquerda", rsuffix="_direita").dropna() # sufixo para não dar erro, dropna para tirar as linhas
  # nas quais o usuário 2 não assistiu ao filme (não é possível haver algum filme que o usuário 1 tenha assistido pois a função - join - funciona assim)

  return diferencas

In [ ]:
def distancia_de_vetores(a,b):
  return np.linalg.norm(a - b)

In [ ]:
def distancia_de_usuarios(usuario_id1, usuario_id2, minimo = 5):
  diferenca = diferencas(usuario_id1, usuario_id2)

  # Tratando dos casos em que os usuarios não viram os mesmos filmes:
  if(len(diferenca) < minimo):
    return None # Já que não há quase nada em comum, vai retornar apenas None

  distancia = distancia_de_vetores(diferenca['nota_esquerda'], diferenca['nota_direita'])
  return(usuario_id1, usuario_id2, distancia)

In [ ]:
distancia_de_usuarios(usuario1ID, usuario2ID)

(611, 612, 3.0413812651491097)

In [ ]:
def porcentagem_distancia(usuario_id1, usuario_id2):
  distancia = distancia_de_usuarios(usuario_id1, usuario_id2)[2] # Pegando somente a distância
  n = len(diferencas(usuario_id1, usuario_id2)) # Quantidade de filmes que ambos assistiram

  if n < 4:
    print("Quantidade insuficiente de filmes em que ambos os usuarios assistiram, isso pode acarretar em perda de credibilidade no prototipo\n")

  dist_max = math.sqrt(math.pow(4.5, 2) * n)

  proporcao = (distancia/dist_max) * 100

  porcentagem = 100 - proporcao
  porcentagem = round(porcentagem, 2)

  print(f"A similaridade entre {usuario_1} e {usuario_2} é de {porcentagem}%")

In [ ]:
porcentagem_distancia(usuario1ID, usuario2ID)

A similaridade entre julia e davi é de 81.25%


# Recomendando outros filmes:
Por último, o protótipo deve sugerir filmes para os dois usuários assistirem. Entendendo acerca de como funcionam os data frames utilizados no protótipo, entendi que a melhor alternativa para fazer essa recomendação dupla é achar o vetor médio dos usuário (tendo em vista que, por exemplo, as notas têm um mínimo e máximo muito bem definidos e outras alternativas poderiam trazer resultados imprecisos devido a essas especificações).

Dessa forma, criei o vetor médio notas_ambos e, a partir dele, calculei as melhores sugestões de filmes, comparando a distância desse vetor médio com os vetores de todos os outros usuários. Assim, a partir dos k usuários mais próximos desse vetor médio (que, por padrão, é 10), há a recomendação dos 5 melhores filmes para os usuários assistirem. Nessa etapa, além de possibilitar a restrição da escolha de busca (diminuindo a quantidade de usuários a serem comparados, porém fazendo com que o programa seja mais rápido em sua busca), também há outros filtros que fazem com que o protótipo seja mais crítico em suas recomendações. Como, por exemplo, a quantidade mínima de filmes para se considerar a distância entre dois vetores, e a quantidade mínima de usuários votantes que cada filme deve ter para ser recomendado.


In [ ]:
notas_do_usuario1 = notas_do_usuario(usuario1ID)
notas_do_usuario2 = notas_do_usuario(usuario2ID)
notas_ambos = pd.concat([notas_do_usuario1, notas_do_usuario2])
notas_ambos = notas_ambos.drop(não_assistidos)
notas_ambos = notas_ambos.groupby("filmeID").mean()[["nota"]]
notas_ambos = notas_ambos.reset_index()
notas_ambos

,filmeID,nota
0,110,1.50
1,364,1.75
2,858,4.75
3,919,2.50
4,1214,3.25
5,1258,3.50
6,1721,1.50
7,1954,3.25
8,2571,4.25
9,2959,5.00


In [ ]:
# Para inserir o vetor médio no data frame "notas"
def insere_notas_médias(dados, usuarioID):
  dados['usuarioID'] = usuarioID
  return pd.concat([notas, dados]) # Concatenando as duas dataframe

In [ ]:
novoID = notas['usuarioID'].max()+1

In [ ]:
notas = insere_notas_médias(notas_ambos, novoID)
notas.tail(20)

,usuarioID,filmeID,nota,momento
9,612,364,2.50,NaN
10,612,1721,2.00,NaN
11,612,1,1.00,NaN
12,612,2571,4.50,NaN
13,612,1214,3.50,NaN
14,612,1258,4.00,NaN
15,612,2959,5.00,NaN
0,613,110,1.50,NaN
1,613,364,1.75,NaN
2,613,858,4.75,NaN


In [ ]:
# Calcula a distância de todos os usuários
def distancia_de_todos(usuario, n = None):
  todos_os_usuarios = notas["usuarioID"].unique()

  if n:
    todos_os_usuarios = todos_os_usuarios[:n] # Vai analisar somente os n primeiros elementos

  distancias = [distancia_de_usuarios(usuario, usuario_id) for usuario_id in todos_os_usuarios]

  distancias = list(filter(None, distancias)) # Cria uma lista com todos os usuarios, tirando todos os None que tinham em distancia

  distancias = pd.DataFrame(distancias, columns = ["voce", "outra_pessoa", "distancia"]) # Transformando em dataframe

  return distancias

In [ ]:
# Retorna os k mais próximos usuários que se "parecem" com o usuário analisado
def knn(voce_id, k_mais_proximos=10, numero_de_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce_id, n = numero_de_usuarios_a_analisar)
  distancias = distancias.sort_values("distancia")
  distancias = distancias.set_index("outra_pessoa").drop(voce_id, errors = 'ignore')
  return distancias.head(k_mais_proximos)

In [ ]:
# Sugere com base no knn
def sugere_para(voce, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_vistos = notas_de_voce.index

  similares = knn(voce, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioID").loc[usuarios_similares]
  recomendacoes = notas_dos_similares.groupby("filmeID").mean()[["nota"]]
  aparicoes = notas_dos_similares.groupby("filmeID").count()[['nota']] # Dataframe que agrupa as notas do similares por cada filme e conta quantas notas cada filme tem

  filtro_minimo = k_mais_proximos / 2 # Parece razoavel que pelo menos metade dos usuarios vizinhos tenham que ter visto o filme para que ele seja recomendado

  recomendacoes = recomendacoes.join(aparicoes, lsuffix="_media_dos_usuarios", rsuffix="_aparicoes_nos_usuarios") # Juntando os dois dataframe pelo indice
  recomendacoes = recomendacoes.query("nota_aparicoes_nos_usuarios >= %.2f" % filtro_minimo)

  recomendacoes = recomendacoes.sort_values("nota_media_dos_usuarios", ascending=False)
  recomendacoes = recomendacoes.drop(filmes_vistos,errors='ignore')

  return recomendacoes.join(filmes)

In [ ]:
sugere_para(novoID).head() # Exibe os 5 melhores filmes para se assistir

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,generos,total de votos,nota media
filmeID,,,,,,
296,4.666667,6,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
527,4.600000,5,Schindler's List (1993),Drama|War,220.0,4.225000
1213,4.333333,6,Goodfellas (1990),Crime|Drama,126.0,4.250000
2858,4.250000,6,American Beauty (1999),Drama|Romance,204.0,4.056373
1198,4.200000,5,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,200.0,4.207500


# Feedback e propostas futuras:
Achei muito interessante fazer esse projeto e, agora, me sinto que sei bem mais acerca da área de Ciência de dados, o que me gerou uma curiosidade maior ainda para explorar mais esse campo.

Realmente acho o projeto importante e pretende "manter ele ativo", pois, como havia mencionado na introdução, realmente não há tantos serviços que se baseiam em recomendações para mais de uma pessoa. Pensando nisso, quero melhorar alguns pontos e talvez hospedá-lo em algum domínio.

Pontos a melhorar:
1. Automatizar a entrada de dados (e aumentá-la)
2. Repensar a forma de avaliar as preferências de filmes (pode ser que alguém não tenha assistido a praticamente nenhum daqueles)
3. Criar outros tipos de recomendação (de gênero específico, por exemplo)
4. Possibilitar a recomendação para mais de 2 pessoas
5. Criar uma interface amigável

